In [ ]:
# LOAD MODULES
# Standard library
import os
import sys

# Third party
from tqdm import tqdm

# NOTE: Your script is not in the root directory. We must hence change the system path
DIR = "../"
os.chdir(DIR)
sys.path.append(DIR)

In [ ]:
import networkx as nx
import pandas as pd

from src.methods.utils.measure_functions_directed import *
from src.data.graph_construction import construct_IBM_graph
from src.utils.graph_processing import graph_community
from src.methods.utils.neighbourhood_functions import GARG_AML_nodeselection

In [ ]:
dataset = "HI-Small"
path = "data/"+dataset+"_Trans.csv"
directed = True

G = construct_IBM_graph(path=path, directed = directed)
G_reduced = graph_community(G, 10)

G_reduced_und = G_reduced.to_undirected()
G_reduced_rev = G_reduced.reverse(copy=True)

nodes = list(G_reduced.nodes)
measure_00_list = []
measure_01_list = []
measure_02_list = []
measure_10_list = []
measure_11_list = []
measure_12_list = []
measure_20_list = []
measure_21_list = []
measure_22_list = []

for node in tqdm(nodes):
    G_ego_second_und = nx.ego_graph(G_reduced_und, node, 2) #Use both incoming and outgoing edges
    G_ego_second = nx.subgraph(G_reduced, G_ego_second_und.nodes)
    G_ego_second_rev = nx.ego_graph(G_reduced_rev, node, 2) #Look at the reverse graph to get the incoming edges

    nodes_0, nodes_1, nodes_2, nodes_ordered = GARG_AML_nodeselection(G_ego_second, node, directed = True, G_ego_second_und = G_ego_second_und, G_ego_second_rev = G_ego_second_rev)

    adj_full = nx.adjacency_matrix(G_ego_second, nodelist=nodes_ordered).toarray()

    size_0 = len(nodes_0)
    size_1 = len(nodes_1)
    size_2 = len(nodes_2)

    measure_00 = measure_00_function(adj_full, size_0)
    measure_01 = measure_01_function(adj_full, size_0, size_1)
    measure_02 = measure_02_function(adj_full, size_0, size_1, size_2)
    measure_10 = measure_10_function(adj_full, size_0, size_1)
    measure_11 = measure_11_function(adj_full, size_0, size_1)
    measure_12 = measure_12_function(adj_full, size_0, size_1, size_2)  
    measure_20 = measure_20_function(adj_full, size_0, size_2)
    measure_21 = measure_21_function(adj_full, size_0, size_1, size_2)
    measure_22 = measure_22_function(adj_full, size_2)

    measure_00_list.append(measure_00)
    measure_01_list.append(measure_01)
    measure_02_list.append(measure_02)
    measure_10_list.append(measure_10)
    measure_11_list.append(measure_11)
    measure_12_list.append(measure_12)
    measure_20_list.append(measure_20)
    measure_21_list.append(measure_21)
    measure_22_list.append(measure_22)

data_dict = {
    "node": nodes, 
    "measure_00": measure_00_list,
    "measure_01": measure_01_list, 
    "measure_02": measure_02_list,
    "measure_10": measure_10_list,
    "measure_11": measure_11_list,
    "measure_12": measure_12_list,
    "measure_20": measure_20_list,
    "measure_21": measure_21_list,
    "measure_22": measure_22_list,
             }

measure_df = pd.DataFrame(data_dict)



In [ ]:
measure_df

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
X = measure_df.drop(columns = ["node"])

In [ ]:
clf = IsolationForest(random_state=1997)
clf.fit(X)

In [ ]:
y_pred = clf.predict(X)

In [ ]:
y_scores = clf.score_samples(X)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(y_scores, bins = 100)
plt.yscale("log")

In [ ]:
y_pred_01 = (1-y_pred)/2
plt.hist(y_pred_01, bins = 100)

In [ ]:
measure_df["anomaly_score"] = y_scores
measure_df["anomaly_pred"] = y_pred_01

In [ ]:
from src.data.pattern_construction import define_ML_labels

In [ ]:
extended_df,pattern_columns = define_ML_labels()

In [ ]:
extended_df.head()

In [ ]:
laundering_from = extended_df[["Account", "Is Laundering"]+pattern_columns].groupby("Account").mean()

In [ ]:
laundering_from.reset_index(inplace=True)

In [ ]:
df_test = pd.merge(
left = measure_df[['node', 'anomaly_score', 'anomaly_pred']],
right = laundering_from, 
left_on="node",
right_on="Account"
)

In [ ]:
df_test.head()

In [ ]:
df_test.corr()

In [ ]:
df_corr = df_test.corr()

In [ ]:
# Extract the node with the highest and lowest anomaly score
# Plot their second order ego network

node_max = df_test.loc[df_test["anomaly_score"].idxmax()]["node"]
node_min = df_test.loc[df_test["anomaly_score"].idxmin()]["node"]

G_ego_max = nx.ego_graph(G_reduced, node_max, 2) #Use both incoming and outgoing edges
G_ego_min = nx.ego_graph(G_reduced, node_min, 2) #Use both incoming and outgoing edges


In [ ]:
nx.draw(G_ego_max, with_labels=True)

In [ ]:
nx.draw(G_ego_min, with_labels=True)

In [ ]:
import numpy as np
r = df_corr["anomaly_score"]['Is Laundering']
N = len(df_test)
t_stat = r*np.sqrt((N-2)/(1-r**2)); print(t_stat)

# Two-tailed t-test
from scipy.stats import t
p = 2*(1-t.cdf(np.abs(t_stat), N-2)); print(p)
if p < 0.05:
    print("Reject null hypothesis")
    print("There is a significant correlation between anomaly score and laundering")
else:
    print("Fail to reject null hypothesis")
    print("There is no significant correlation between anomaly score and laundering")